In [ ]:
!wget https://s3-eu-west-1.amazonaws.com/fever.public/wiki-pages.zip
!wget https://github.com/facebookresearch/DrQA/archive/master.zip
!wget https://s3-eu-west-1.amazonaws.com/fever.public/train.jsonl

In [ ]:
!unzip wiki-pages.zip
!unzip master.zip

In [ ]:
# setup drqa env
!cd DrQA && python3 setup.py develop
!pip install -r DrQA-master/requirements.txt

# build wiki_docs sqlite db
!cd DrQA/ && python3 scripts/retriever/build_db.py ../wiki-pages ../wiki_docs.db

In [ ]:
import json
import itertools
import sqlite3

In [ ]:
# get all sqlite documents in the train pieces
all_articles = set()
with open('train.jsonl', 'r') as fp:
    claims = []
    for jsonl in fp.readlines():
        jsoned = json.loads(jsonl)
        for evidence in itertools.chain(*jsoned['evidence']):
            if evidence[2] is not None:
                all_articles.add(evidence[2])
        claims.append(jsoned)

print(len(claims))
print(len(all_articles))

In [69]:
# join article ids to get selected from db
articles_joined = ', '.join(map(lambda art: "'{}'".format(art.replace("'", "''")), all_articles))
sql_articles = 'select * from documents where id in ({})'.format(articles_joined)

In [ ]:
# get all articles
conn = sqlite3.connect('wiki_docs.db')
with sqlite3.connect('wiki_docs.db') as conn:
    cur = conn.cursor()
    cur.execute(sql_articles)
    article_rows = cur.fetchall()


In [ ]:
#create skimmed documents database and insert train data documents
def db_create_table(db_filepath):
    create_table = '''
        create table if not exists documents (
            id text primary key,
            text text not null
        )
    '''
    with sqlite3.connect(db_filepath) as conn:
        cur = conn.cursor()
        cur.execute(create_table)

def db_insert_rows(db_filepath, rows):
    insert = '''
        insert or ignore into documents
        values {};
    '''
    tupl_str_art = list(map(lambda art: "('{}', '{}')".format(art[0].replace("'", "''"), art[1].replace("'", "''")), rows))

    insert_values = insert.format(', '.join(tupl_str_art))

    with sqlite3.connect(db_filepath) as conn:
        cur = conn.cursor()
        cur.execute(insert_values)
        cur.execute('select count(1) from documents')
        count = cur.fetchone()
    return count

db_create_table('wiki_docs_skimmed.db')
db_insert_rows('wiki_docs_skimmed.db', article_rows)

In [ ]:
# add extra documents from articles
# CANNOT SEED will be random every time
extra_docs = 9 * len(all_articles)
with sqlite3.connect('wiki_docs.db') as conn:
  cur = conn.cursor()
  cur.execute('SELECT * FROM documents ORDER BY RANDOM() LIMIT {};'.format(extra_docs))
  unknown_arts = cur.fetchall()

db_insert_rows('wiki_docs_skimmed.db', unknown_arts)


In [ ]:
# build tfidf for new skimmed document set
!cd DrQA/ && python3 scripts/retriever/build_tfidf.py ../wiki_docs_skimmed.db ../